# Ausgangscode von Aufgabe 2

In [ ]:
import cv2
import numpy as np

# Open the image file
img = cv2.imread('Gesicht.jpg')

# Set up the mouse callback function
def draw_circle(event,x,y,flags,param):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img,(x,y),3,(0,0,255),-1)    #punkt erzeugen
        points.append((x,y))    #Punkte die angelickt wurden speichern
        print(points)
        if len(points) > 2:
            cv2.polylines(img, [np.array(points)], False, (0, 255, 0), thickness=2) #image, liste aus Punkten in zweier Tupel, soll das Polygon geschlossen werden, Farbe, dicke Linie

def Cutout():
    pts = np.array(points)   #np Array aus den Punkten des Polygon erstellen
    mask = np.zeros(img.shape[:2], np.uint8) #Maske erzeugen aus den Punkten
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA) #Konturen zeichnen

    cutImg = cv2.bitwise_and(img, img, mask=mask)  #Bildbereich ausschneiden
    cv2.imwrite("Cut Image.png", cutImg)    #Ausgeschnittenes Bild speichern

    # Erstelle ein neues Bild mit einem leeren Alphakanal
    cutImgWithAlpha = cv2.cvtColor(cutImg, cv2.COLOR_BGR2BGRA)
    cutImgWithAlpha[:, :, 3] = mask #Maske auf das neue Bild mir Alphakanal anwenden

    # Erstelle ein neues Bild mit einem leeren Alphakanal im Hintergrund
    background = np.zeros((img.shape[0], img.shape[1], 4), dtype=np.uint8)

    # Invertiere die Maske, um das ausgeschnittene Gesicht transparent zu machen
    mask_inv = cv2.bitwise_not(mask)

    # Verwende die Maske, um den Hintergrund transparent zu machen
    background[mask_inv == 255] = [0, 0, 0, 0]

    # Füge das ausgeschnittene Gesicht zum Hintergrund hinzu
    finalImg = cv2.add(background, cutImgWithAlpha)
    cv2.imwrite("Final Image.png", finalImg)

# Neues Ausgabe Window erzeugen
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)
points = [] #Liste um Punkte des Polygones zu speichern

# Loop das die Tasten abfrägt
while True:
    cv2.imshow('image',img)  #Ausgabe des Image in einer Unendlichen Schleife
    
    if cv2.waitKey(1) == ord('q'):  #Abbruch mit der Taste q
        break

    if cv2.waitKey(1) == ord('c'):  #Ausschneiden und Speichern des Bereichs mit der Taste c 
        Cutout()

cv2.destroyAllWindows()   #Anwendung beenden

# Funktionierende Augenerkennung bei Gesicht.png

In [8]:
import cv2
import numpy as np
import math

#Berechnen des Winkels zwischen den beiden Punkten        
def CalcAngle(cords):
    print("Cords: ",cords)
    deltaX = cords[0][0] - cords[1][0]
    deltaY = cords[0][1] - cords[1][1]
    print(deltaX,deltaY)
    tan = deltaY/deltaX
    alpha = math.degrees(math.tan(tan))
    return alpha

# Laden der beiden Bilder
image1 = cv2.imread('CutFace.png')
image2 = cv2.imread('Gesicht.png')

# Erstellen der neuen Ausgabe-Matrix
img = np.zeros((max(image1.shape[0], image2.shape[0]), image1.shape[1]+image2.shape[1], 3), dtype='uint8')

# Einfügen des ersten Bildes
img[0:image1.shape[0], 0:image1.shape[1]] = image1

# Einfügen des zweiten Bildes
img[0:image2.shape[0], image1.shape[1]:image1.shape[1]+image2.shape[1]] = image2

# Überprüfen, ob das Bild erfolgreich eingelesen wurde
if img is None:
    print('Fehler: Konnte das Bild nicht einlesen')
    exit()

# Überprüfen, ob das Bild einen Alpha-Kanal hat
if len(img.shape) == 3 and img.shape[2] == 4:
    print("Das Bild hat einen Alphakanal")
    alpha = img[:, :, 3]
    img_without_alpha = img[:, :, :3]
else:
    print("Das Bild hat keinen Alphakanal")
    alpha = None
    img_without_alpha = img

# Umwandeln in Graustufenbild
gray = cv2.cvtColor(img_without_alpha, cv2.COLOR_BGR2GRAY)

# Haar-Cascade-Datei für Augen laden
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Erkennen der Augen
eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
print("Augen: ",eyes)

#Augen Positionen herausfiltern (x,y) und sortieren von augen Position links nach rechts
eyesPos = list()
for i in range(len(eyes)):
    eyesPos.append((eyes[i][0],eyes[i][1]))

print(sorted(eyesPos))

angleLeft = CalcAngle(eyesPos[:2])
angleRight = CalcAngle(eyesPos[2:])

print(angleLeft, angleRight)

# Zeichnen von Rechtecken um die erkannten Augen
for (x, y, w, h) in eyes:
    cv2.rectangle(img_without_alpha, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Zusammenführen von Alpha-Kanal und Bildoutput
# Wenn Bild Alphakanal hat
if alpha is not None:
    result = np.dstack((img_without_alpha, alpha))
# Wenn Bild keinen Alphakanal hat
else:
    result = img_without_alpha

# Anzeigen des Ergebnisses
cv2.imshow('Augen erkannt', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


Das Bild hat keinen Alphakanal
Augen:  [[258 105  48  48]
 [869 105  49  49]
 [319 109  46  46]
 [931 109  46  46]]
[(258, 105), (319, 109), (869, 105), (931, 109)]
Cords:  [(258, 105), (869, 105)]
-611 0
Cords:  [(319, 109), (931, 109)]
-612 0
-0.0 -0.0


QObject::moveToThread: Current thread (0x24885f0) is not the object's thread (0x2860a40).
Cannot move to target thread (0x24885f0)

QObject::moveToThread: Current thread (0x24885f0) is not the object's thread (0x2860a40).
Cannot move to target thread (0x24885f0)

QObject::moveToThread: Current thread (0x24885f0) is not the object's thread (0x2860a40).
Cannot move to target thread (0x24885f0)

QObject::moveToThread: Current thread (0x24885f0) is not the object's thread (0x2860a40).
Cannot move to target thread (0x24885f0)

QObject::moveToThread: Current thread (0x24885f0) is not the object's thread (0x2860a40).
Cannot move to target thread (0x24885f0)

QObject::moveToThread: Current thread (0x24885f0) is not the object's thread (0x2860a40).
Cannot move to target thread (0x24885f0)

QObject::moveToThread: Current thread (0x24885f0) is not the object's thread (0x2860a40).
Cannot move to target thread (0x24885f0)

QObject::moveToThread: Current thread (0x24885f0) is not the object's thread

# Rotierung und Skalierung fuktionieren

In [61]:
import cv2
import numpy as np
import math

def GetEyes():
    # Umwandeln in Graustufenbild
    gray = cv2.cvtColor(img_without_alpha, cv2.COLOR_BGR2GRAY)

    # Haar-Cascade-Datei für Augen laden
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

    # Erkennen der Augen
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
    print("Augen: ",eyes)
    return eyes

#Berechnen des Winkels zwischen den beiden Punkten        
def CalcAngleAndSpacing(cords):
    print("Cords: ",cords)
    deltaX = cords[0][0] - cords[1][0]
    deltaY = cords[0][1] - cords[1][1]
    print("dela X:", deltaX, "delta Y: ", deltaY)
    tan = deltaY/deltaX
    alpha = math.degrees(math.tan(tan))
    spacing = math.sqrt((deltaX ** 2 + deltaY ** 2))
    print("Abstand: ", spacing)
    return alpha, spacing

#Rotieren und Skalieren des Bildes mit berechnetem Winkel 
def RotateAndScale(image, left, right):
    deltaAlpha = rigth[0] - left[0]
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, deltaAlpha, 1.0)
    image = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)

    #Bild Sklaieren auf den gleichen Augenabstand des linken Bildes
    scale_percent = 100 - (1-left[1]/right[1])*100 #scale des Bildes in % 
    print("Skalierung", scale_percent)
    width = int(image.shape[1] * scale_percent / 100) #Berechung der Breite
    height = int(image.shape[0] * scale_percent / 100) #Berechnung der Höhe
    dim = (width, height) #Tupel aus neuer Skalierung
    result = cv2.resize(image, dim, interpolation = cv2.INTER_AREA) #Resize Methode
    cv2.imwrite("Result.png", result)
    return result


# Laden der beiden Bilder
image1 = cv2.imread('GesichtFrau.jpeg')
image2 = cv2.imread('CutFace.png', cv2.IMREAD_UNCHANGED)

# Erstellen der neuen Ausgabe-Matrix
img = np.zeros((max(image1.shape[0], image2.shape[0]), image1.shape[1]+image2.shape[1], 3), dtype='uint8')

# Einfügen des ersten Bildes
img[0:image1.shape[0], 0:image1.shape[1]] = image1

# Einfügen des zweiten Bildes
img[0:image2.shape[0], image1.shape[1]:image1.shape[1]+image2.shape[1]] = image2

# Überprüfen, ob das Bild erfolgreich eingelesen wurde
if img is None:
    print('Fehler: Konnte das Bild nicht einlesen')
    exit()

# Überprüfen, ob das Bild einen Alpha-Kanal hat
if len(img.shape) == 3 and img.shape[2] == 4:
    print("Das Bild hat einen Alphakanal")
    alpha = img[:, :, 3]
    img_without_alpha = img[:, :, :3]
else:
    print("Das Bild hat keinen Alphakanal")
    alpha = None
    img_without_alpha = img

#Augen Position erfassen
eyes = GetEyes()

#Augen Positionen herausfiltern (x,y) und sortieren von augen Position links nach rechts
eyesPos = list()
for i in range(len(eyes)):
    eyesPos.append((eyes[i][0],eyes[i][1]))

print(sorted(eyesPos))

left = CalcAngleAndSpacing(eyesPos[:2])
print(left)
angleLeft = left[0]
spacingLeft = left[1]
rigth = CalcAngleAndSpacing(eyesPos[2:])
angleRight = rigth[0]
spacingRigth = rigth[1]

print("Angle Left: ",angleLeft,"Angle Rigth: ", angleRight)

#Rechtes Bild auf den Winkel drehen und Skalieren des linken Bildes
rotatet = RotateAndScale(image2, left, rigth)
cv2.imshow('Test', rotatet)

# Zeichnen von Rechtecken um die erkannten Augen
for (x, y, w, h) in eyes:
    cv2.rectangle(img_without_alpha, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Zusammenführen von Alpha-Kanal und Bildoutput
# Wenn Bild Alphakanal hat
if alpha is not None:
    result = np.dstack((img_without_alpha, alpha))
# Wenn Bild keinen Alphakanal hat
else:
    result = img_without_alpha

# Anzeigen des Ergebnisses
cv2.imshow('Augen erkannt', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


ValueError: could not broadcast input array from shape (408,612,4) into shape (408,612,3)

In [1]:
test = [1,2,3,4]
print(test[:2])
print(test[2:])

[1, 2]
[3, 4]


In [100]:
# Laden der beiden Bilder
image1 = cv2.imread('GesichtFrau.jpeg')
image2 = cv2.imread('CutFace.png')

# Überprüfen, ob das obere Bild erfolgreich geladen wurde
if image1 is None:
    print('Fehler beim Laden des oberen Bildes.')
else:
    # Überlappen der beiden Bilder
    h, w = image1.shape[:2]
    x_offset = 100
    y_offset = 50
    for i in range(h):
        for j in range(w):
            if x_offset+j < image2.shape[1] and y_offset+i < image2.shape[0]:
                # Ersetzen von cv2.add() durch eine einfache Elementweise Addition
                image2[y_offset+i][x_offset+j] = image1[i][j] + image2[y_offset+i][x_offset+j]


    # Anzeigen der 
    cv2.imshow('Ausgabe', image2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread

# Finale Version mit allen Funktionen

In [45]:
import cv2
import numpy as np
import math

def GetEyes():
    # Umwandeln in Graustufenbild
    gray = cv2.cvtColor(img_without_alpha, cv2.COLOR_BGR2GRAY)

    # Haar-Cascade-Datei für Augen laden
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

    # Erkennen der Augen
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
    print("Augen: ",eyes)
    return eyes

#Berechnen des Winkels zwischen den beiden Punkten        
def CalcAngleAndSpacing(cords):
    print("Cords: ",cords)
    deltaX = cords[0][0] - cords[1][0]
    deltaY = cords[0][1] - cords[1][1]
    print("dela X:", deltaX, "delta Y: ", deltaY)
    tan = deltaY/deltaX
    alpha = math.degrees(math.tan(tan))
    spacing = math.sqrt((deltaX ** 2 + deltaY ** 2))
    print("Abstand: ", spacing)
    return alpha, spacing

#Rotieren und Skalieren des Bildes mit berechnetem Winkel 
def RotateAndScale(image, left, right):
    deltaAlpha = rigth[0] - left[0]
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, deltaAlpha, 1.0)
    image = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)

    #Bild Sklaieren auf den gleichen Augenabstand des linken Bildes
    scale_percent = 100 - (1-left[1]/right[1])*100 #scale des Bildes in % 
    print("Skalierung", scale_percent)
    width = int(image.shape[1] * scale_percent / 100) #Berechung der Breite
    height = int(image.shape[0] * scale_percent / 100) #Berechnung der Höhe
    dim = (width, height) #Tupel aus neuer Skalierung
    result = cv2.resize(image, dim, interpolation = cv2.INTER_AREA) #Resize Methode
    cv2.imwrite("Result.png", result)
    return result

#Überlappen der Beiden Bilder, damit die Augen aufeinander liegen
def Overlap(image1, image2):
    #Größe Breite und Größte Höhe herausfinden
    if image1.shape[0] < image2.shape[0]:
        y = image2.shape[0]
    else:
        y = image1.shape[0]

    if image1.shape[1] < image2.shape[1]:
        x = image2.shape[1]
    else:
        x = image1.shape[1]

    print("X: ", x, " Y: ", y)
    #Neues np Array anlegen, dass die größte Breite und die Größte Höhe hat
    result = np.zeros((y, x, 3), dtype=np.uint8)

    #Offset mit dem das GesichtFrau Bild verschoben werden soll
    xOffset = 427
    yOffset = 239

    #Durchiterrieren aller Pixel um das Finale Bild generieren zu lassen result ist ein Bild mit RGB Kanal
    for i in range(x):
        for j in range(y):
            #try fängt indizierungen auserhalb des erlaubten Bereichs ab
            try:
                if image2[j][i][3] == 0 and j-yOffset < 0 and i-xOffset < 0:
                    result[j][i]  = [image1[j-yOffset][i-xOffset][0],image1[j-yOffset][i-xOffset][1],image1[j-yOffset][i-xOffset][2]]

                else:
                    result[j][i]  = [image2[j][i][0],image2[j][i][1],image2[j][i][2]]
            except:
                result[j][i]  = [image2[j][i][0],image2[j][i][1],image2[j][i][2]]
    return result


# Laden der beiden Bilder
image1 = cv2.imread('GesichtFrau.jpeg')
image2 = cv2.imread('CutFace.png')

# Erstellen der neuen Ausgabe-Matrix
img = np.zeros((max(image1.shape[0], image2.shape[0]), image1.shape[1]+image2.shape[1], 3), dtype='uint8')

# Einfügen des ersten Bildes
img[0:image1.shape[0], 0:image1.shape[1]] = image1

# Einfügen des zweiten Bildes
img[0:image2.shape[0], image1.shape[1]:image1.shape[1]+image2.shape[1]] = image2

# Überprüfen, ob das Bild erfolgreich eingelesen wurde
if img is None:
    print('Fehler: Konnte das Bild nicht einlesen')
    exit()

# Überprüfen, ob das Bild einen Alpha-Kanal hat
if len(img.shape) == 3 and img.shape[2] == 4:
    print("Das Bild hat einen Alphakanal")
    alpha = img[:, :, 3]
    img_without_alpha = img[:, :, :3]
else:
    print("Das Bild hat keinen Alphakanal")
    alpha = None
    img_without_alpha = img

#Augen Position erfassen
eyes = GetEyes()

#Augen Positionen herausfiltern (x,y) und sortieren von augen Position links nach rechts
eyesPos = list()
for i in range(len(eyes)):
    eyesPos.append((eyes[i][0],eyes[i][1]))

print(sorted(eyesPos))

left = CalcAngleAndSpacing(eyesPos[:2])
print(left)
angleLeft = left[0]
spacingLeft = left[1]
rigth = CalcAngleAndSpacing(eyesPos[2:])
angleRight = rigth[0]
spacingRigth = rigth[1]

print("Angle Left: ",angleLeft,"Angle Rigth: ", angleRight)

#CutFace nun mit Alphakanal laden
image2 = cv2.imread('CutFace.png', cv2.IMREAD_UNCHANGED)

#Rechtes Bild auf den Winkel drehen und Skalieren des linken Bildes
rotatet = RotateAndScale(image2, left, rigth)

#Bilder übereinander schieben
finalImage = Overlap(image1, rotatet)
cv2.imshow('Final', finalImage)

# Zeichnen von Rechtecken um die erkannten Augen
for (x, y, w, h) in eyes:
    cv2.rectangle(img_without_alpha, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Zusammenführen von Alpha-Kanal und Bildoutput
# Wenn Bild Alphakanal hat
if alpha is not None:
    result = np.dstack((img_without_alpha, alpha))
# Wenn Bild keinen Alphakanal hat
else:
    result = img_without_alpha

# Anzeigen des Ergebnisses
cv2.imshow('Augen erkannt', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


Das Bild hat keinen Alphakanal
Augen:  [[ 95  38  44  44]
 [ 47  61  35  35]
 [534 107  47  47]
 [594 109  45  45]]
[(47, 61), (95, 38), (534, 107), (594, 109)]
Cords:  [(95, 38), (47, 61)]
dela X: 48 delta Y:  -23
Abstand:  53.225933528685054
(-29.768142962234492, 53.225933528685054)
Cords:  [(534, 107), (594, 109)]
dela X: -60 delta Y:  -2
Abstand:  60.03332407921454
Angle Left:  -29.768142962234492 Angle Rigth:  1.9105669869269606
Skalierung 88.66064697409215
X:  542  Y:  361


QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc7

In [77]:
import cv2

# Laden der beiden Bilder
image1 = cv2.imread('Result.png', cv2.IMREAD_UNCHANGED)
image2 = cv2.imread('GesichtFrau.jpeg', cv2.IMREAD_UNCHANGED)

# Überprüfen, ob beide Bilder erfolgreich geladen wurden
if image1 is None or image2 is None:
    print('Fehler beim Laden der Bilder.')
else:
    # Überprüfen, ob das obere Bild einen Alphakanal hat
    if len(image1.shape) == 3 and image1.shape[2] == 4:
        has_alpha_channel = True
    else:
        has_alpha_channel = False

    # Überlappen der beiden Bilder
    h1, w1 = image1.shape[:2]
    h2, w2 = image2.shape[:2]
    x_offset = 100
    y_offset = 50
    for i in range(h1):
        for j in range(w1):
            if x_offset+j < w2 and y_offset+i < h2:
                if has_alpha_channel and image1[i][j][3] == 0:
                    # Alphakanal von Bild1 ist 0, Bild2-Pixel übernehmen
                    image2[y_offset+i][x_offset+j] = image2[y_offset+i][x_offset+j]
                else:
                    # Alphakanal von Bild1 ist nicht 0, Bild1-Pixel übernehmen
                    image2[y_offset+i][x_offset+j] = image1[i][j]

    # Anzeigen der Ausgabe
    cv2.imshow('Ausgabe', image2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



ValueError: could not broadcast input array from shape (4,) into shape (3,)

In [102]:
# Laden der beiden Bilder
image1 = cv2.imread('GesichtFrau.jpeg')
image2 = cv2.imread('CutFace.png')

# Überprüfen, ob das obere Bild erfolgreich geladen wurde
if image1 is None:
    print('Fehler beim Laden des oberen Bildes.')
else:
    # Überlappen der beiden Bilder
    h, w = image1.shape[:2]
    x_offset = 100
    y_offset = 50
    for i in range(h):
        for j in range(w):
            if x_offset+j < image2.shape[1] and y_offset+i < image2.shape[0]:
                if all(c == 0 for c in image2[y_offset+i][x_offset+j][:3]):
                    # Pixel in image2 ist schwarz, Addition der Pixel
                    image2[y_offset+i][x_offset+j] = image1[i][j] + image2[y_offset+i][x_offset+j]
                else:
                    # Pixel in image2 ist nicht schwarz, Image2 Pixel wird übernommen
                    image2[y_offset+i][x_offset+j] = image2[y_offset+i][x_offset+j]

    # Anzeigen der Ausgabe
    cv2.imshow('Ausgabe', image2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread (0x16cc6e0).
Cannot move to target thread (0x16f3630)

QObject::moveToThread: Current thread (0x16f3630) is not the object's thread

In [38]:
import cv2
import numpy as np

# Laden der beiden Bilder
image1 = cv2.imread('GesichtFrau.jpeg')
image2 = cv2.imread('Result.png', cv2.IMREAD_UNCHANGED)

if image1.shape[0] < image2.shape[0]:
    y = image2.shape[0]
else:
    y = image1.shape[0]

if image1.shape[1] < image2.shape[1]:
    x = image2.shape[1]
else:
    x = image1.shape[1]

print("X: ", x, " Y: ", y)
result = np.zeros((y, x, 3), dtype=np.uint8)

xOffset = 400
yOffset = 250

for i in range(x):
    for j in range(y):
        try:
            if image2[j][i][3] == 0 and j-yOffset < 0 and i-xOffset < 0:
                result[j][i]  = [image1[j-yOffset][i-xOffset][0],image1[j-yOffset][i-xOffset][1],image1[j-yOffset][i-xOffset][2]]

            else:
                result[j][i]  = [image2[j][i][0],image2[j][i][1],image2[j][i][2]]
        except:
            result[j][i]  = [image2[j][i][0],image2[j][i][1],image2[j][i][2]]

print(w,h)
print(result)
cv2.imshow('Ausgabe', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

X:  542  Y:  361
542 361
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc7

In [6]:
import numpy as np

# Ein leeres NumPy-Array mit der Form (100, 100, 4) erstellen
image = np.zeros((100, 100, 3), dtype=np.uint8)

# Alle Pixel auf blau setzen (RGBA-Farbpalette: (0, 0, 255, 255))
image[:, :, 0] = 255    #blau
image[:, :, 1] = 0      #grün
image[:, :, 2] = 0      #rot

image[1][0] = [0,255,0]  #Zeile, Spalte

# Das Bild mit OpenCV anzeigen
cv2.imshow('Blaues Bild', image)
cv2.waitKey(0)
cv2.destroyAllWindows()



QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc74b0).
Cannot move to target thread (0x14e6c90)

QObject::moveToThread: Current thread (0x14e6c90) is not the object's thread (0xdc7

ValueError: cannot reshape array of size 150975 into shape (183,275,3,256)